In [1]:
# this is where we showcase our final model on a small sample of the data (once we are done creating our best model)

In [ ]:
"""
What to do:
1) load the best model from a pickle file
2) create a small (less than 5 MB) image data array from a folder of select test images
3) run the model and showcase the performance
4) make sure the entire notebook takes less than a minute to run

"""